In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from IPython.display import display, Markdown, Latex

In [4]:
from pathlib import Path
home = str(Path.home())

In [5]:
import sys
sys.path.insert(0,f'{home}/knowledgelib/')

In [6]:
import pyknowledge

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [8]:
import scipy.io
import pandas as pd
import numpy as np

mat = scipy.io.loadmat("/disk/metabric/BRCA1View20000.mat")

In [9]:
gene_labels = open("gene_labels.txt").read().split("\n")
pd.Series(gene_labels)

0          SCGB2A2
1            MUCL1
2          SCGB1D2
3              PIP
4        LOC648852
           ...    
19995    LOC642453
19996    LOC652100
19997    LOC646050
19998    LOC644912
19999    LOC652294
Length: 20000, dtype: object

In [10]:
gene_labels = [g[0] for g in mat['gene'][0]]
df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
# df['id'] = mat['id'][0]
df['target'] = mat['targets']
df['Subtype'] = df.target.map({1:'Basal',2:'HER2+',3:'LumA',4:'LumB',5:'Normal Like',6:'Normal'})
df.head()

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,target,Subtype
0,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6,Normal
1,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4,LumB
2,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4,LumB
3,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5,Normal Like
4,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094,3,LumA


In [11]:
lum_genes = pd.read_csv('Luminal_B_101_Gene_List.csv')

genes_df = df.loc[:,df.columns.isin(lum_genes["Gene"])]

## Here is where we would select difference scalers

In [12]:
scaler = None

if scaler is None:
    scaler = StandardScaler()
    scaler.fit(genes_df)

genes_df_scaled = pd.DataFrame(scaler.transform(genes_df),index=genes_df.index,columns=genes_df.columns).fillna(0)

In [13]:
genes_df_scaled.head()

,CCND1,FAM83H,C17orf28,FAM83D,SRGAP1,UBE2C,BOP1,BRF2,KIFC2,TROAP,...,RAD54B,NFKBIL2,ZBTB10,PSMC3IP,LETM2,BRCA1,FAM82B,MYBL2,FRS2,CIT
0,-0.100659,-3.189314,-1.163106,-1.368472,0.197607,-2.160346,-1.268345,0.235924,-0.933807,-1.898709,...,-1.410917,-0.539807,-0.990863,-1.394209,-1.683707,-1.760110,0.504558,-0.916205,-1.437369,1.309344
1,-0.396379,1.923176,0.505856,-0.638042,1.477128,0.040077,2.057732,2.492817,2.267095,-0.076725,...,0.275690,0.731575,-0.143936,-0.736753,-0.378679,-0.281442,1.326211,1.086842,-0.746088,-0.877594
2,0.153928,2.076008,0.727836,-0.948505,0.592500,-0.527023,1.303371,0.617340,1.742926,-0.027667,...,-0.095986,-1.137496,-1.343864,-0.062959,0.889682,1.095956,2.489593,0.528456,0.021510,-0.872164
3,0.764704,0.830027,-0.425847,-0.142911,0.458682,0.967930,1.843483,-0.298788,1.120019,1.767628,...,-0.692801,1.483901,-1.242918,0.485040,-0.336100,1.149192,0.498185,-1.617987,0.403393,2.220044
4,1.547597,0.188069,1.024349,-0.520472,0.484273,0.318252,-0.363769,2.339653,0.595944,0.124801,...,-0.001107,-0.621344,-1.815289,1.112204,1.060896,1.191882,-1.432317,2.028444,0.153414,-0.619178


In [15]:
data = genes_df_scaled.join(df[['Subtype']])
pca_data = data.set_index('Subtype').loc['LumB']
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(pca_data)

PCA(n_components=2)

In [27]:
pca.explained_variance_ratio_

array([0.2258795 , 0.10070977])

In [25]:
data_transformed = pd.DataFrame(pca.transform(data.drop('Subtype',axis=1))).join(df[['Subtype']])
data_transformed.columns = ["PC1","PC2","Subtype"]
data_transformed

,PC1,PC2,Subtype
0,-10.887005,-0.150642,Normal
1,6.661073,-3.643411,LumB
2,1.576173,-2.152591,LumB
3,-0.211867,-3.208371,Normal Like
4,-0.547350,2.891889,LumA
...,...,...,...
2128,-3.016126,0.228740,LumA
2129,-8.931000,-2.081556,Normal Like
2130,-4.210068,-0.552935,HER2+
2131,-5.430923,0.124227,Normal Like


In [26]:
import altair as alt
alt.Chart(data_transformed).mark_circle(size=60).encode(
    x='PC1',
    y='PC2',
    color='Subtype:N'
    #tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
)#.interactive()

alt.Chart(...)

### Look at the genes individually

In [ ]:
source = genes_df_scaled.join(df[['Subtype','target']]).melt(id_vars=['Subtype','target'])
source.columns = ["Subtype","target","Gene","Value"]
counts = source.groupby('Subtype')['target'].count().to_frame()
counts.columns = ['Count']
source = source.set_index('Subtype').join(counts).reset_index()
# use the url approach to keep notebook smaller
url = 'Luminal_B_101_to_graph_fig1.json'
pyknowledge.plot.prepare_url(source,url)
source

In [26]:
pyknowledge.plot.histogram_1(url,'Value','Subtype','Gene')

alt.Chart(...)

## Knowledge Overview
These genes are deemed important based on previous research into the LuminalB subtype. The pattern is unspecified and is a set of genes. As previously mentioned, the relationship between pairs of samples is the input to the modified cross-entropy loss function. This requires defining a distance between samples. We define the distance as the number of the number of LumB genes within one standard deviation of one another.

In [28]:
# get us ready to parallize distance
ncores = 7
pair_ixs = pyknowledge.common.get_pair_inxs(df.shape[0])
chunks_pair_ixs = list(pyknowledge.common.divide_chunks(pair_ixs,pyknowledge.common.calc_len_chunk(len(pair_ixs),ncores)))

In [29]:
from joblib import Parallel, delayed

In [37]:
helper = lambda distance_func: pd.concat(Parallel(n_jobs=-1)(delayed(pyknowledge.distance.chunk_distance)(genes_df_scaled,chunk,distance_func) for chunk in chunks_pair_ixs))

In [38]:
DL1 = helper(pyknowledge.distance.L1)

In [39]:
DL2 = helper(pyknowledge.distance.L2)

In [40]:
DFSIGN = helper(pyknowledge.distance.FSIGN)

In [41]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [43]:
Ds = {'DL1':DL1,'DL2':DL2,'DFSIGN':DFSIGN}

In [44]:
distances = {}
for key in Ds.keys():
    distances[key] = pd.DataFrame(list(Ds[key]['distance']),columns=[key],index=Ds[key].index)

In [46]:
for key in distances.keys():
    distances[key] = pyknowledge.distance.remove_self_ref(distances[key]).dropna()

In [50]:
for key in distances.keys():
    distances[key] = pyknowledge.common.add_labels(df,distances[key])

In [51]:
distances[key].head()

DFSIGN  label1       label2         label1_label2
index1 index2                                                     
0      1       0.422222  Normal         LumB         Normal - LumB
       2       0.433333  Normal         LumB         Normal - LumB
       3       0.466667  Normal  Normal Like  Normal - Normal Like
       4       0.366667  Normal         LumA         Normal - LumA
       5       0.344444  Normal        HER2+        Normal - HER2+

### Prepare our distances for plotting

In [71]:
subtypes = df.Subtype.unique()
for subtype in subtypes:
    for key in distances.keys():
        source = pyknowledge.plot.prepare_source1(df,distances[key],subtype,subtypes)
        group_counts = source.groupby('label1_label2')[key].count().to_frame()
        group_counts.columns = ['Count']
        source = source.set_index('label1_label2').join(group_counts).reset_index()
        url = f"{subtype}_{key}_hist.json"
        source.to_json(url,orient='records')

In [72]:
source.head()

,label1_label2,index1,index2,DFSIGN,label1,label2,label1_label2_ordered,Count
0,Basal - Basal,31,35,0.622222,Basal,Basal,Basal - Basal,54285
1,Basal - Basal,31,39,0.522222,Basal,Basal,Basal - Basal,54285
2,Basal - Basal,31,59,0.644444,Basal,Basal,Basal - Basal,54285
3,Basal - Basal,31,63,0.622222,Basal,Basal,Basal - Basal,54285
4,Basal - Basal,31,65,0.688889,Basal,Basal,Basal - Basal,54285


In [73]:
subtypes = df.Subtype.unique()
for key in distances.keys():
    for subtype in ["LumA","LumB"]:
        url = f"{subtype}_{key}_hist.json"
        g = alt.Chart(url).transform_calculate(
            pct='1 / datum.Count'
        ).mark_area(
            opacity=0.3,
            interpolate='step'
        ).encode(
            alt.X('%s:Q'%key, bin=alt.Bin(maxbins=50)),
            alt.Y('sum(pct):Q', axis=alt.Axis(format='%'),stack=None),
            alt.Color('label1_label2:N',title='Label - Label')
        )
        display(Markdown(f'### {key}: {subtype}'))
        display(g)

### DL1: LumA

alt.Chart(...)

### DL1: LumB

alt.Chart(...)

### DL2: LumA

...

alt.Chart(...)

### DFSIGN: LumA

alt.Chart(...)

### DFSIGN: LumB

alt.Chart(...)

# Old below

In [ ]:
alt.Chart(source.sample(n=10000)).mark_area(
    opacity=0.3,
    interpolate='step'
).encode(
    alt.X('DL2:Q', bin=alt.Bin(maxbins=50)),
    alt.Y('count()', stack=None),
    alt.Color('label1_label2:N')
)

In [ ]:
g.save('LumB_LumB_distances.png')

In [ ]:
print('test')

In [ ]:
constant = 'LumA'
distance1_12 = get_distance(distance1,constant,constant)
for n in df['Subtype'].unique():
    if n != constant:
        distance1_12 = distance1_12.append(get_distance(distance1,constant,n))

alt.Chart(distance1_12.reset_index()).mark_area(
    opacity=0.3,
    interpolate='step'
).encode(
    alt.X('D1:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('label1_label2:N')
)

In [ ]:
constant = 'LumB'
distance1_12 = get_distance(distance1,constant,constant)
for n in df['Subtype'].unique():
    if n != constant:
        distance1_12 = distance1_12.append(get_distance(distance1,constant,n))

alt.Chart(distance1_12.reset_index()).mark_area(
    opacity=0.3,
    interpolate='step'
).encode(
    alt.X('D1:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('label1_label2:N')
)

In [ ]:
D2 = pd.concat(Parallel(n_jobs=-1)(delayed(chunk_distance)(chunk,distance2) for chunk in chunks_pair_ixs))

## Set some minimum values for sparsity

In [ ]:
min_value = genes_df_scaled.shape[1]*0.3 # Choose a minimum value
min_value

In [ ]:
D2.plot.hist()

In [ ]:
D2.unstack()

In [ ]:
D2_visual = D2.copy() # or D2
D2_visual.loc[D2.distance <= min_value] = 0 # just to make it more sparse
D2_visual=D2_visual.fillna(0).unstack()
D2_visual.index.name='index'
D2_visual.columns = D2_visual.index
D2_visual = D2_visual/genes_df_scaled.shape[1]
D2_visual.to_csv(f'graphs/D2_gt_{min_value}.csv')

In [ ]:
!ls graphs

In [ ]:
min_value

In [ ]:
D2_visual

In [ ]:
min_value = genes_df_scaled.shape[1]*0.6 # Choose a minimum value
min_value

In [ ]:
D1_visual = D1.copy() # or D2
D1_visual.loc[D1.distance <= min_value] = 0 # just to make it more sparse
D1_visual = D1_visual.fillna(0).unstack()
D1_visual.index.name='index'
D1_visual.columns = D1_visual.index
D1_visual.to_csv(f'graphs/D1_gt_{min_value}.csv')
D1_visual#.dropna()

## Test out the knowledge functionality

In [ ]:
import knowledge

In [ ]:
k = knowledge.Knowledge()
k.distances

In [ ]:
#import tensorflow as tf
def cosine_sim(logits):
    x_ = tf.expand_dims(logits, 0)
    y_ = tf.expand_dims(logits, 1)
    xN = tf.sqrt(tf.reduce_sum(tf.reshape((x_[None]*y_[:,None])*0+x_[None]**2, [-1, logits.shape[1]]),axis=1))
    yN = tf.sqrt(tf.reduce_sum(tf.reshape((x_[None]*y_[:,None])*0+y_[:,None]**2, [-1, logits.shape[1]]),axis=1))
    #z = tf.reshape(cosine_loss(y_true, y_pred), [-1, logits.shape[1]]), reduction_indices=[1])),axis=0)    
    z = tf.reduce_sum(tf.reshape((x_[None]*y_[:,None]), [-1, logits.shape[1]]),axis=1)/(xN*yN)
    return z

#sess = tf.Session()
#logits = tf.constant(genes_df_scaled.values[:3,:], dtype=tf.float32)
#A = sess.run(cosine_sim(logits))

In [ ]:
A

In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

pd.DataFrame(cosine_similarity(genes_df_scaled.values[:3,:])).stack()

This is looking good. We just need to do all combinations and then set the index. 

In [ ]:
np.reshape(A,(3,3))

In [ ]:



D_visual.dropna()

In [ ]:
from IPython.display import Image

import networkx as nx

def save(A,file="graph.png"):
    g = A.draw(format=file.split(".")[-1], prog='dot')
    open(file,"wb").write(g)
    return Image(g)

G = nx.Graph()
temp = D_visual.loc[D_visual.distance > 0].reset_index()
for n in np.unique(list(temp.index1) + list(temp.index2)):
    c = df.loc[n,'color']
    G.add_node(n,color=c,style='filled',fillcolor=c)

for index,values in D_visual.loc[D_visual.distance > 0].iterrows():
    n1,n2 = index
    G.add_edge(n1,n2,weight=genes_df_scaled.shape[1]-values.loc['distance'])

In [ ]:
pos = nx.drawing.nx_agraph.graphviz_layout(G, prog='dot')
A = nx.nx_agraph.to_agraph(G)
A.graph_attr["rankdir"] = "LR"
# draw it in the notebook
save(A)

The additional terms to the loss function are broken into label-label, label-unknown, and unknown-unknown. We will model the label-label case first where mw stands for the match weight, mm stands for the mismatch weight.

<pre>
label1  label2 w
---------------------------
LumB    LumB    mw
Unknown LumB    0|mmw
LumB    Unknown 0|mmw
Unknown Unknown 0
</pre>
If both samples are LumB subtype, then the weight to the loss function is mw > 0. If only a single sample is LumB subtype, then we may also set the mmw = 0 or mmw > 0. If you set mmw > 0, then you may use labeled and unlabelled.

What I would like to test:
1. Does this improve performance and/or convergence of DeepType (at least during the supervised learning phase)
2. When running repeated experiments with varying random seed and/or parameters, do we have more consistency of results? 


### Let's work on a sample model

In [ ]:
nsample = 10
genes_df_scaled_sampled = genes_df_scaled.sample(n=nsample,random_state=0)
samples = genes_df_scaled_sampled.join(df[['Subtype','target']])
(100*samples[['CCND1','FAM83H','C17orf28','FAM83D']]).round().astype(int).join(df[['Subtype','target']])

In [ ]:
import itertools

def get_diff(samples,y,cls):
    diff_samples = pd.DataFrame(columns = list(genes_df_scaled_sampled.columns)+["Subtype_1","Subtype_2","ix1","ix2"])
    diff_samples = diff_samples.set_index(["ix1","ix2"])
    for ix1,ix2 in itertools.permutations(samples.index,2):
        if ix1 == ix2:
            continue
        new_sample = samples.loc[ix1,genes_df_scaled_sampled.columns] - samples.loc[ix2,genes_df_scaled_sampled.columns]
        new_sample = new_sample.to_frame().T
        #new_sample["label_1"] = samples.loc[ix1,"label"]
        #new_sample["label_2"] = samples.loc[ix2,"label"]
        new_sample["Subtype_1"] = y.loc[ix1] #samples.loc[ix1,"Subtype"]
        new_sample["Subtype_2"] = y.loc[ix2] #samples.loc[ix2,"Subtype"]
        new_sample["ix1"] = ix1
        new_sample["ix2"] = ix2
        new_sample = new_sample.set_index(["ix1","ix2"])
        diff_samples = diff_samples.append(new_sample)
        
    diff_samples['target'] = "MatchIgnore" # Case 4
    diff_samples.loc[(diff_samples['Subtype_1'] != cls) & (diff_samples['Subtype_2'] == cls),'target'] = "Mismatch1"
    diff_samples.loc[(diff_samples['Subtype_1'] == cls) & (diff_samples['Subtype_2'] != cls),'target'] = "Mismatch2"
    diff_samples.loc[(diff_samples['Subtype_1'] == cls) & (diff_samples['Subtype_2'] == cls),'target'] = "Match"
    diff_samples['target'] = diff_samples['target'].astype('category')
    return diff_samples

<pre>
ID label1 label2 f(id1,id2)
---------------------------
1  LumB   LumB   Match
2  Other  LumB   Mismatch1
3  LumB   Other  Mismatch2
4  Other  Other  MatchIgnore
</pre>

### Now construct the model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, RepeatedStratifiedKFold, StratifiedShuffleSplit
import imblearn

oversample = imblearn.over_sampling.RandomOverSampler()
undersample = imblearn.under_sampling.RandomUnderSampler()

#X,y = diff_samples[genes_df_scaled_sampled.columns], diff_samples['target']
X,y = samples[genes_df_scaled_sampled.columns], samples['Subtype']

#X, y = oversample.fit_resample(X_orig, y_orig)
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats = 2, random_state=1)

#print("f1_micro")
#print(cross_val_score(clf, X, y, cv=cv, scoring='f1_micro'))
#print("f1_macro")
#print(cross_val_score(clf, X, y, cv=cv, scoring='f1_macro'))
#print("f1_weighted")
#print(cross_val_score(clf, X, y, cv=cv, scoring='f1_weighted'))

In [ ]:
from sklearn.metrics import classification_report
clf = RandomForestClassifier(max_depth=2, random_state=0)
from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier()

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

labels = y.unique()
cnt = 1

all_scores_df = []
random_states = [0]
for random_state in random_states:
    cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=random_state)
    scores = {}
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        print(f'Round {cnt}')
        for cls in labels:
            diff_samples = get_diff(X_train,y_train,cls)
            Xdiff = diff_samples.drop(["target","Subtype_1","Subtype_2"],axis=1)
            ydiff = diff_samples['target']
            Xone = Xdiff.loc[ydiff == "Match"]
            
            clf = OneClassSVM(gamma='auto')
            #clf = LocalOutlierFactor(n_neighbors=2,novelty=True)
            clf.fit(Xone)

            #X_over, y_over = oversample.fit_resample(X_train, y_train)
            #X_under, y_under = undersample.fit_resample(X_train, y_train)
            #clf.fit(X_under,y_under)

            diff_samples_test = get_diff(X_test,y_test,cls)
            Xdiff_test = diff_samples_test.drop(["target","Subtype_1","Subtype_2"],axis=1)
            ydiff_test = diff_samples_test['target']
            decf = clf.decision_function(Xdiff_test)
            new_data =  pd.DataFrame(decf,index=ydiff_test.index).join(ydiff_test)
            if cls not in scores:
                scores[cls] = new_data
            else:
                scores[cls].append(new_data)
        cnt+=1

In [ ]:
Xone

In [ ]:
scores['LumA']

In [ ]:
scores['LumA'].loc[scores['LumA'].target=='Match'].mean()

In [ ]:
scores['LumA'].loc[scores['LumA'].target!='Match'].mean()

In [ ]:
scores_df_target.loc[scores_df_target.Subtype!='LumB']

In [ ]:
means = pd.DataFrame(columns=["Match","Mismatch","Subtype"]).set_index("Subtype")
for label in labels:
    mismatch = scores_df_target.loc[scores_df_target.Subtype!=label][label].mean()
    match = scores_df_target.loc[scores_df_target.Subtype==label][label].mean()
    means = means.append(pd.Series([match,mismatch],name=label,index=means.columns))
means

In [ ]:
scores_df_target.loc[scores_df_target.Subtype=='Basal']['Basal'].mean()

In [ ]:
clf.predict_proba(diff_samples[genes_df_scaled_sampled.columns].iloc[[0]])

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(genes_df_scaled)

print(pca.explained_variance_ratio_)

print(pca.singular_values_)

In [ ]:
scores = pca.transform(genes_df_scaled)
scores = pd.DataFrame(scores,columns=["PC1","PC2"])
scores['Subtype'] = df.label.map({1:'Basal',2:'HER2+',3:'LumA',4:'LumB',5:'Normal Like',6:'Normal'})

In [ ]:
scores

In [ ]:
import altair as alt
points = alt.Chart(scores).mark_point().encode(
    x='PC1:Q',
    y='PC2:Q',
    color='Subtype:N'
)
points

In [ ]:
import tensorflow as tf
def cosine_sim(logits):
    x_ = tf.expand_dims(logits, 0)
    y_ = tf.expand_dims(logits, 1)
    xN = tf.sqrt(tf.reduce_sum(tf.reshape((x_[None]*y_[:,None])*0+x_[None]**2, [-1, logits.shape[1]]),axis=1))
    yN = tf.sqrt(tf.reduce_sum(tf.reshape((x_[None]*y_[:,None])*0+y_[:,None]**2, [-1, logits.shape[1]]),axis=1))
    #z = tf.reshape(cosine_loss(y_true, y_pred), [-1, logits.shape[1]]), reduction_indices=[1])),axis=0)    
    z = tf.reduce_sum(tf.reshape((x_[None]*y_[:,None]), [-1, logits.shape[1]]),axis=1)/(xN*yN)
    return z

In [ ]:
import tensorflow as tf
def cosine_sim(logits):
    x_ = tf.expand_dims(logits, 0)
    y_ = tf.expand_dims(logits, 1)
    xN = tf.sqrt(tf.reduce_sum(tf.reshape((x_[None]*y_[:,None])*0+x_[None]**2, [-1, logits.shape[1]]),axis=1))
    yN = tf.sqrt(tf.reduce_sum(tf.reshape((x_[None]*y_[:,None])*0+y_[:,None]**2, [-1, logits.shape[1]]),axis=1))
    #z = tf.reshape(cosine_loss(y_true, y_pred), [-1, logits.shape[1]]), reduction_indices=[1])),axis=0)    
    z = tf.reduce_sum(tf.reshape((x_[None]*y_[:,None]), [-1, logits.shape[1]]),axis=1)/(xN*yN)
    return z

sess = tf.Session()
logits = tf.constant(genes_df_scaled.values, dtype=tf.float32)
A = sess.run(cosine_sim(logits))

In [ ]:
m = int(np.sqrt(len(A)))
A = np.reshape(A,(m,m))

In [ ]:
pd.DataFrame(A).to_csv('graph_Luminal_B_101_BRCA1View20000.csv')

In [ ]:
import networkx as nx
G=nx.from_numpy_matrix(A)

In [ ]:
from IPython.display import Image

def save(A):
    open("graph.png","wb").write(A.draw(format='png', prog='dot'))

pos = nx.drawing.nx_agraph.graphviz_layout(G, prog='neato')
#print(edge_labels)
# Modify node fillcolor and edge color.
#D.node_attr.update(color='blue', style='filled', fillcolor='yellow')
#D.edge_attr.update(color='blue', arrowsize=1)
A = nx.nx_agraph.to_agraph(G)
A.graph_attr["rankdir"] = "LR"
# draw it in the notebook
save(A)